In [2]:
import pandas as pd
import awswrangler as awr
import openpyxl


In [3]:


path_to_query = r"C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\listagem_mestra_ajustada.sql"

with open (path_to_query,'r') as reading:
    query_readed = reading.read()

df = awr.athena.read_sql_query(query_readed,'silver')

In [4]:
df.info()
len(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131295 entries, 0 to 131294
Data columns (total 25 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   cooperativa      131295 non-null  string
 1   unidade          131291 non-null  string
 2   cliente          131295 non-null  string
 3   matricula        131295 non-null  Int64 
 4   conjunto         131295 non-null  Int64 
 5   coverage         131295 non-null  Int64 
 6   beneficio        131295 non-null  string
 7   motorista        13630 non-null   string
 8   id_placa         88393 non-null   Int64 
 9   placa            108190 non-null  string
 10  chassi           88393 non-null   string
 11  renavam          88380 non-null   string
 12  ativacao         131289 non-null  object
 13  inicio_vig       131289 non-null  object
 14  fim_vig          131289 non-null  object
 15  status           131295 non-null  string
 16  status_coverage  131295 non-null  string
 17  benefits_i

131295

placas totais

In [5]:
df['placa'].drop_duplicates().count()

31681

seguro de vida geral

In [6]:
df_seguro_de_vida = df[df['benefits_id'].isin([6,28,44])]
len(df_seguro_de_vida)

#print(len(df_seguro_de_vida))
#df_seguro_de_vida.head()
#print(len(df_seguro_de_vida.drop_duplicates(subset='placa')))
#df_seguro_de_vida['placa'].drop_duplicates().count()

12196

seguro de vida de motorista

In [7]:
df_seguro_de_vida_motorista = df_seguro_de_vida[df_seguro_de_vida['faixa_valor'].str.contains('MOTORISTA', na=False)]
len(df_seguro_de_vida_motorista)

9669

In [8]:
df_cascos_veiculo = df[df['benefits_id'].isin([2, 24, 40])]
len(df_cascos_veiculo)

17195

In [10]:
df_cascos_veiculo_empresa = df_cascos_veiculo[df_cascos_veiculo['cooperativa']=='Viavante']
df_cascos_veiculo_empresa.head()

,cooperativa,unidade,cliente,matricula,conjunto,coverage,beneficio,motorista,id_placa,placa,...,status,status_coverage,benefits_id,categoria,tipo_categoria,faixa_valor,tabela_ativa,tipo_veiculo,tipo,e_mail
40658,Viavante,UNIDADE GUARAPUAVA,AUGUSTO ZIMERMANN,4574,6587,48338,CASCO (VEÍCULO),<NA>,14253674,BCV4C92,...,ATIVO,ATIVO,40,CAVALO MECÂNICO,CP: 4% - MAN - TODOS OS ANOS,"De R$325.000,01 até R$350.000,00",N,Pesado - Cavalo,Novo,<NA>
40660,Viavante,TRANSDESK DIGITAL LTDA,Erichyson Oliveira dos Santos,4575,6588,48349,CASCO (VEÍCULO),<NA>,13699183,ARX2J46,...,ATIVO,ATIVO,40,CAVALO MECÂNICO,CP: 4% - VOLKS - TODOS OS ANOS,"De R$125.000,01 até R$150.000,00",N,Pesado - Cavalo,Novo,<NA>
40664,Viavante,UNIDADE LONDRINA,AJL PECAS AGRICOLAS LTDA,1501,6592,48383,CASCO (VEÍCULO),<NA>,6024549,MRK6H52,...,ATIVO,ATIVO,40,CAVALO MECÂNICO,CP: 4% - SCANIA - TODOS OS ANOS,"De R$175.000,01 até R$200.000,00",N,Pesado - Cavalo,Novo,<NA>
40677,Viavante,MF - MICRO FRANQUEADO JULIANO DE COSTA,SCHAFER TRANSPORTES E COMERCIO DE MADEIRAS LTDA,4578,6596,48414,CASCO (VEÍCULO),<NA>,14177930,RLG3H48,...,ATIVO,ATIVO,40,CAMINHÃO,CP: 4% - TODAS AS MARCAS - TODOS OS ANOS,"De R$400.000,01 até R$425.000,00",N,Pesado - Bitruck,Novo,<NA>
40678,Viavante,MF - MICRO FRANQUEADO JULIANO DE COSTA,SCHAFER TRANSPORTES E COMERCIO DE MADEIRAS LTDA,4578,6596,48411,CASCO (VEÍCULO),<NA>,14177916,MHL1809,...,ATIVO,ATIVO,40,3 QUARTOS,CP: 4% - TODAS AS MARCAS - TODOS OS ANOS,"De R$150.000,01 até R$175.000,00",N,Pesado - Três Quartos,Novo,<NA>


In [19]:
df_cascos_sem_motorista = df_cascos_veiculo[df_cascos_veiculo['motorista'].notna()]
len(df_cascos_sem_motorista)

0

In [23]:
df_com_motorista = df[df['motorista'].notna()]
df_com_motorista['beneficio'].unique()
#df_empresa = df_com_motorista[df_com_motorista['cooperativa']=='Viavante']
#df_empresa.head()

<StringArray>
[                 'SEGURO DE VIDA',                    'RASTREAMENTO',
                      'RASTREADOR', 'RASTREADOR REBOQUE/SEMIRREBOQUE',
                        'ANUIDADE', 'T4S - RASTREADOR / IMOBILIZADOR',
             'INSERIDO NA APOLICE',                          'VIDROS']
Length: 8, dtype: string

In [25]:
df_filter1= df_com_motorista[
    (df_com_motorista['cooperativa'] == 'Viavante') &
    (df_com_motorista['beneficio'] != 'SEGURO DE VIDA')
]

df_filter1.head()

,cooperativa,unidade,cliente,matricula,conjunto,coverage,beneficio,motorista,id_placa,placa,...,status,status_coverage,benefits_id,categoria,tipo_categoria,faixa_valor,tabela_ativa,tipo_veiculo,tipo,e_mail
40832,Viavante,UNIDADE SINOP,2R TRANSPORTES COMERCIO E REPRESENTACAO LTDA,4652,6744,49742,RASTREADOR REBOQUE/SEMIRREBOQUE,FERNANDO VELOSO LEAO JUNIOR,<NA>,NGW3C98,...,ATIVO,ATIVO,51,PADRÃO,PADRÃO,TRACKERLOG MAX,S,<NA>,Novo,<NA>
40967,Viavante,UNIDADE SINOP,2R TRANSPORTES COMERCIO E REPRESENTACAO LTDA,4652,6811,50443,RASTREADOR,LUCAS AMARO MARIANO,15258487,NUE0A10,...,ATIVO,ATIVO,46,PADRÃO,PADRÃO,TRACKERLOG MAX CAVALO,S,Pesado - Cavalo,Novo,<NA>
41449,Viavante,UNIDADE SINOP,OFICINA AUTOMOTIVA VIA NORTE LTDA,1886,5656,40790,RASTREADOR REBOQUE/SEMIRREBOQUE,PAULINO RAMIREZ,<NA>,RRP1I63,...,ATIVO,ATIVO,51,PADRÃO,PADRÃO,TRACKERLOG MAX,S,<NA>,Novo,<NA>
42362,Viavante,UNIDADE SINOP,LEANDRO FERREIRA DE MELO,5420,8226,61914,RASTREADOR,LEANDRO FERREIRA DE MELO,15924796,NDE2E95,...,ATIVO,ATIVO,46,PADRÃO,PADRÃO,TRACKERLOG MAX CAVALO,S,Pesado - Bitruck,Novo,<NA>
42369,Viavante,UNIDADE SINOP,LUCIANO SICHINEL AGROVETERINARIA,5423,8231,61952,RASTREADOR REBOQUE/SEMIRREBOQUE,SILVIO OLIVEIRA DOMINGUES,<NA>,QVX6E18,...,ATIVO,ATIVO,51,PADRÃO,PADRÃO,TRACKER RF CARRETA,S,<NA>,Novo,<NA>


base_analise

In [4]:

#excluir coluna de e-mail pois está diferenciando na análise
#df = df.drop('e_mail', axis=1)
df = df.drop_duplicates()

#pegar apenas os conjuntos obrigatorios de seguro de vida
beneficios_conjuntos = df.groupby(['conjunto', 'cooperativa'])['benefits_id'].unique()
conjuntos_validos = beneficios_conjuntos[(beneficios_conjuntos.apply(lambda x: {2,6}.issubset((x))))|
                                         (beneficios_conjuntos.apply(lambda x: {24,28}.issubset((x))))|
                                         (beneficios_conjuntos.apply(lambda x: {40,44}.issubset((x))))].index

df_obrigatorio = df[df.set_index(['conjunto','cooperativa']).index.isin(conjuntos_validos)].copy()

#criar colunas para posterior agregação
df_obrigatorio['casco'] = df_obrigatorio['benefits_id'].apply(lambda x: x if (x == 2 or x == 24 or x == 40) else pd.NA)
#df_obrigatorio['seg_vida'] = df_obrigatorio['benefits_id'].apply(lambda x: x if (x==6 or x==28 or x==44) else pd.NA)


df_obrigatorio['seg_vida_motorista'] = df_obrigatorio.apply(lambda row: 
                                                  row['benefits_id'] if (row['benefits_id'] in [6,28,44] and 'MOTORISTA' in row['faixa_valor'])
else pd.NA, axis=1)

df_obrigatorio['seg_vida_geral'] = df_obrigatorio.apply(lambda row: 
                                                  row['benefits_id'] if (row['benefits_id'] in [6,28,44] and 'MOTORISTA' not in row['faixa_valor'])
else pd.NA, axis=1)

#filtrando apenas os que fazem sentido para a análise (casco e seguro de vida)
df_obrigatorio_filtrado = df_obrigatorio[df_obrigatorio['benefits_id'].isin([2,6,24,28,40,44])]

In [5]:
df_obrigatorio_filtrado.head()

,cooperativa,unidade,cliente,matricula,conjunto,coverage,beneficio,motorista,id_placa,placa,...,categoria,tipo_categoria,faixa_valor,tabela_ativa,tipo_veiculo,tipo,e_mail,casco,seg_vida_motorista,seg_vida_geral
0,Stcoop,UNIDADE SINOP,FABIO TURRA JAEGER,21573,13608,84648,CASCO (VEÍCULO),<NA>,17310,RLJ1C15,...,CAVALO,LC: 100% - CP: 4% - SCANIA - 2024 ATÉ 2021,"De R$600.000,01 até R$625.000,00",S,Cavalo,RenovaÃ§Ã£o,<NA>,24,<NA>,<NA>
1,Stcoop,UNIDADE CASCAVEL,RICARDO ROSA AMORIM EIRELI,21578,13610,84655,CASCO (VEÍCULO),<NA>,19062,QAY6H04,...,CAVALO,LC: 100% - CP: 4% - SCANIA - 2024 ATÉ 2021,"De R$625.000,01 até R$650.000,00",S,Cavalo,RenovaÃ§Ã£o,<NA>,24,<NA>,<NA>
5,Stcoop,UNIDADE RONDONÓPOLIS,VILMAR ALEXANDRE DA SILVA TRANSPORTES,11931,13612,84672,SEGURO DE VIDA,VILMAR ALEXANDRE DA SILVA,<NA>,<NA>,...,PADRÃO,PADRÃO,MOTORISTA - Seguro de Vida via Rep. ou Rep. do...,N,<NA>,RenovaÃ§Ã£o,<NA>,<NA>,28,<NA>
8,Stcoop,UNIDADE CURITIBA,A. SAID TRANSPORTES LTDA - ME,6400,13616,84691,CASCO (VEÍCULO),<NA>,18914,BDV1A44,...,CAVALO,LC: 100% - CP: 4% - DAF E MAN - 2024 ATÉ 2019,"De R$350.000,01 até R$375.000,00",S,Cavalo,RenovaÃ§Ã£o,<NA>,24,<NA>,<NA>
10,Stcoop,UNIDADE ITAJAI,SANDRO LUIZ VIEIRA,2312,13618,84709,SEGURO DE VIDA,SANDRO LUIZ VIEIRA,<NA>,<NA>,...,PADRÃO,PADRÃO,MOTORISTA - Seguro de Vida via Rep. ou Rep. do...,N,<NA>,RenovaÃ§Ã£o,<NA>,<NA>,28,<NA>


In [7]:
df_obrigatorio_filtrado['seg_vida_motorista'].count()

9611

conjuntos_analise

In [14]:


df_analise = df_obrigatorio_filtrado.groupby(['cooperativa','unidade','matricula','conjunto','e_mail']).agg(
    qtd_seg_vida_motorista = ('seg_vida_motorista', 'count'),
    qtd_casco = ('casco', 'count'),
    qtd_seg_vida_geral = ('seg_vida_geral', 'count')
).reset_index()

#df_analise.drop_duplicates('e_mail', keep='first')
df_analise.sort_values(by='conjunto', ascending=False)

# df_analysis=df_analise[(df_analise['qtd_motorista']!=df_analise['qtd_casco'])|
#                       (df_analise['qtd_motorista']!=df_analise['qtd_seg_vida'])|
#                       (df_analise['qtd_casco']!=df_analise['qtd_seg_vida'])]

df_analysis = df_analise[df_analise['qtd_seg_vida_motorista']<df_analise['qtd_casco']]

#df_analysis = df_analise[row, ['cooperativa','unidade','matricula','conjunto','qtd_motoristas','qtd_cascos','qtd_segvida']]

# file_path  = r'C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\conjuntos_analise.xlsx'
# df_analysis.to_excel(file_path, engine = 'openpyxl', index = False)

# file_path  = r'C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\conjuntos_variados.xlsx'
# df_analise.to_excel(file_path, engine = 'openpyxl', index = False)



parametros de e-mail

In [15]:
#-----------------LOAD
#Function to send e-mails

#p_email,

def enviar_email(p_unidade, p_empresa, tabela_html):    
    outlook = win32.Dispatch('outlook.application')  #integrating python with outlook 

    #creating e-mail variable to use
    email = outlook.CreateItem(0)

    #configuring the e-mail - ALTEREI PARA TESTE
    email.To = "dados13@grupounus.com.br; suelen.martucci@transdesk.com.br"
    email.Subject = "[PENDÊNCIA DE SEGURO DE VIDA - MOTORISTAS] - Análise de Dados"
    email.HTMLBody = f"""

    <p>Prezado(a),</p>

    <p>A {p_unidade} ({p_empresa}) possui pendências de cadastramento de seguro de vida <b>para motoristas</b>.</p>

    <p>A quantidade de coberturas concernentes ao casco é maior do que a quantidade de coberturas de seguro de vida <b>para motoristas</b>.</p>

    <p>{tabela_html}</p>

    <p>Favor, cadastrar os seguros de vida <b>para motoristas</b> remanescentes para cômputo na base de dados da empresa, obrigado!</p>

    <p>Atenciosamente,</p>

    <p>Equipe Análise de Dados - Grupo Unus</p>

    <p><i>Este é um e-mail automático, por favor não responda</i></p>

    """

    email.Send()

    print("Email enviado")

função envio email real

In [ ]:
df_analysis3 = df_analysis2.drop('qtd_motorista', axis=1)
df_email = df_analysis3[['cooperativa','unidade','e_mail']].drop_duplicates('e_mail', keep='first')

for _, row in df_email.iterrows():
    iEmail = row['e_mail']
    iUnidade = row['unidade']
    iCooperativa = row['cooperativa']

    df_filtrado = df_analysis3[(df_analysis3['unidade']==iUnidade) & (df_analysis3['cooperativa']==iCooperativa)]

    if df_filtrado.empty:
        print(f"Nenhum dado encontrado para {iUnidade} - {iCooperativa}. E-mail não enviado.")
        continue  

    df_tabela = df_filtrado.to_html(index=False, border=1)

    # if __name__ == '__main__':
    #     enviar_email(iEmail, iUnidade, iCooperativa, df_tabela)

    print(f"E-mail enviado para {iEmail}: {iUnidade}({iCooperativa})")



função envio e-mail teste

In [16]:

# df_analysis['e_mail']='dados13@grupounus.com.br'
df_analysis.drop(columns='qtd_seg_vida_geral', axis=1, inplace=True)
df_email = df_analysis[['cooperativa','unidade','e_mail']].drop_duplicates('unidade', keep='first')
df_email

for _, row in df_email.iterrows():
    iEmail = row['e_mail']
    iUnidade = row['unidade']
    iCooperativa = row['cooperativa']

    df_filtrado = df_analysis[(df_analysis['unidade']==iUnidade) & (df_analysis['cooperativa']==iCooperativa)]

    if df_filtrado.empty:
        print(f"Nenhum dado encontrado para {iUnidade} - {iCooperativa}. E-mail não enviado.")
        continue  

    df_tabela = df_filtrado.to_html(index=False, border=1, classes = "tabela")

    df_tabela = f"""
    <style>
        .tabela {{
            width: 100%;
            border-collapse: collapse;
        }}
        .tabela th, .tabela td {{
            text-align: center;  
            padding: 8px;
            border: 1px solid black;
        }}
    </style>
    {df_tabela}
    """
#iEmail, 
    if __name__ == '__main__':
        enviar_email(iUnidade, iCooperativa, df_tabela)

    print(f"E-mail enviado para {iEmail}: {iUnidade}({iCooperativa})")


C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_31672\616413508.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analysis.drop(columns='qtd_seg_vida_geral', axis=1, inplace=True)


Email enviado
E-mail enviado para contasapagar.unus@gmail.com: TS CONSULTORIA EM TRANSPORTES LTDA(Stcoop)
Email enviado
E-mail enviado para unidade.cascavel@transdesk.com.br: UNIDADE CASCAVEL(Stcoop)
Email enviado
E-mail enviado para unidade.curitiba@transdesk.com.br: UNIDADE CURITIBA(Stcoop)
Email enviado
E-mail enviado para unidade.sinop@transdesk.com.br: UNIDADE SINOP(Stcoop)
Email enviado
E-mail enviado para unidade.vilhena@transdesk.com.br: UNIDADE VILHENA(Stcoop)
Email enviado
E-mail enviado para unidade.campogrande@transdesk.com.br: UNIDADE CAMPO GRANDE(Viavante)
Email enviado
E-mail enviado para unidade.carazinho@transdesk.com.br: UNIDADE CARAZINHO(Viavante)
Email enviado
E-mail enviado para unidade.cuiaba@transdesk.com.br: UNIDADE CUIABA(Viavante)
Email enviado
E-mail enviado para unidade.curitiba-vendaonline@transdesk.com.br: UNIDADE CURITIBA - VENDA ONLINE(Viavante)
Email enviado
E-mail enviado para unidade.dourados@transdesk.com.br: UNIDADE DOURADOS(Viavante)
Email enviado
